In [1]:
# default_exp main

In [1]:
# export
import os
import sys
import pandas as pd  
import numpy as np
from tqdm import tqdm  
import scipy.sparse as sp
import pickle
from catboost import Pool, CatBoostClassifier
from collections import defaultdict
import math

In [ ]:
# export
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import math
import pickle
import numpy as np
class PickleWrapper(object):

    @classmethod
    def loadFromFile(cls, file, mode='rb'):
        with open(file, mode) as f:
            return pickle.load(f)

    @classmethod
    def dump2File(cls, o, file, mode='wb'):
        with open(file, mode) as f:
            pickle.dump(o, f)
def get_sim_item(df_, user_col, item_col, use_iif=False): 

    df = df_.copy()
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))
    
    user_time_ = df.groupby(user_col)['time'].agg(list).reset_index() # 引入时间因素
    user_time_dict = dict(zip(user_time_[user_col], user_time_['time']))
    
    sim_item = {}  
    item_cnt = defaultdict(int)  # 商品被点击次数
    for user, items in tqdm(user_item_dict.items()):  
        for loc1, item in enumerate(items):  
            item_cnt[item] += 1  
            sim_item.setdefault(item, {})  
            for loc2, relate_item in enumerate(items):  
                if item == relate_item:  
                    continue  
                t1 = user_time_dict[user][loc1] # 点击时间提取
                t2 = user_time_dict[user][loc2]
                sim_item[item].setdefault(relate_item, 0)  
                if not use_iif:  
                    if loc1-loc2>0:
                        sim_item[item][relate_item] += 1 * 0.7 * (0.8**(loc1-loc2-1)) * (1 - (t1 - t2) * 10000) / math.log(1 + len(items)) # 逆向
                    else:
                        sim_item[item][relate_item] += 1 * 1.0 * (0.8**(loc2-loc1-1)) * (1 - (t2 - t1) * 10000) / math.log(1 + len(items)) # 正向
#                 else:  
#                     sim_item[item][relate_item] += 1 / math.log(1 + len(items))  

    sim_item_corr = sim_item.copy() # 引入AB的各种被点击次数  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
#             sim_item_corr[i][j] = cij / ((item_cnt[i] * item_cnt[j]) ** 0.2)  
            sim_item_corr[i][j] = cij / (np.log(item_cnt[i]+1) * np.log(item_cnt[j]+1))
    return sim_item_corr, user_item_dict  


def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):  
    '''
    input:item_sim_list, user_item, uid, 500, 50
    # 用户历史序列中的所有商品均有关联商品,整合这些关联商品,进行相似性排序
    '''
    rank = {}  
    interacted_items = user_item_dict[user_id] 
    interacted_items = interacted_items[::-1]
    for loc, i in enumerate(interacted_items):  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij * (0.7**loc) 

    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  

# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

now_phase = 9
train_path = '../data/underexpose_train'  
test_path = '../data/underexpose_test'  


whole_click = pd.DataFrame()  
r = [] 
for c in range(7, now_phase + 1):  
    recom_item = []  
    print('phase:', c)  
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'], converters={'user_id':str, 'item_id':str})  
    click_test = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c,c), header=None,  names=['user_id', 'item_id', 'time'], converters={'user_id':str, 'item_id':str})  

    all_click = click_train.append(click_test)  
    whole_click = whole_click.append(all_click)  
    whole_click = whole_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
    whole_click = whole_click.sort_values('time')

    item_sim_list, user_item = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=False)  

    for i in tqdm(click_test['user_id'].unique()):  
        rank_item = recommend(item_sim_list, user_item, i, 500, 500)  
        rank = 1
        for j in rank_item:  
            
            recom_item.append([i, j[0], j[1], rank])  
            rank += 1
            
    dft = pd.DataFrame(recom_item)

    dft.columns = 'user_id item_id_pred score rank'.split()
    r.append(dft)
    
PickleWrapper.dump2File(r, '../user_data/tmp_data/r_list_itemcf_op_test_789.pkl')

In [ ]:
# export

def load_click_data_per_phase(now_phase):
    """
    """
    train_path = '../data/underexpose_train'
    test_path = '../data/underexpose_test'

     
    all_click_df = []
    for c in range(now_phase + 1):  
        print(f'phase: {c}')  
        cols_str = 'user_id item_id time'.split()
        click_train1 = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'], converters={c: str for c in cols_str})  
        click_test1 = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c, c), header=None,  names=['user_id', 'item_id', 'time'], converters={c: str for c in cols_str}) 
        test_qtime1 = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_qtime-{}.csv'.format(c, c), header=None,  names=['user_id','time'], converters={c: str for c in cols_str})  
        click_test1_val = click_test1.sort_values(['user_id', 'time']).drop_duplicates(subset=['user_id'],keep='last')
        
        click_test1 = click_test1[~click_test1.index.isin(click_test1_val.index)]
        all_click = click_train1.append(click_test1).drop_duplicates().sort_values('time')
        
        all_click_df.append((all_click, click_test1_val, test_qtime1))
        print(f'all_click: {all_click.shape}, click_test1_val: {click_test1_val.shape}, test_qtime1: {test_qtime1.shape}')
    return all_click_df

In [ ]:
# export
def get_item_data():
    train_item_df = pd.read_csv('../data/underexpose_train/underexpose_item_feat.csv', sep=r',\s+|,\[|\],\[',
                                names=['item_id']+list(range(256)),
                                converters={'item_id':str})
    train_item_df.iloc[:, -1] = train_item_df.iloc[:, -1].str.replace(']', '').map(float)
    return train_item_df

In [ ]:
# export

            
def build_co_occurance_matrix(items_list,
                              window=9999,
                              penalty1=False,
                              penalty2=False,
                              penalty3=1,
                              penalty4=False,
                              save_dir=None):
    from multiprocessing.dummy import Pool, Lock
    from collections import Counter

    pool = Pool()
    mutex = Lock()
    item_list_flat = [ii for i in items_list for ii in i]
    item_num_dict = dict(Counter(item_list_flat))  # 每个item出现次数的字典

    items = pd.Series(list(item_num_dict.keys()))
    item2id = pd.Series(items.index, items)

    n_items = items.shape[0]
    print(f'n_items: {n_items}')
    train_data_matrix = sp.lil_matrix((n_items, n_items), dtype=np.float)
    def t(items_):
#     for items_ in tqdm(items_list):
        for i, item in enumerate(items_):
            for j, related_item in enumerate(items_):
                distance = np.abs(i-j)
                if (item != related_item) and (distance<window):
                    vt = 1
                    if penalty1:
#                         print('对距离惩罚，距离越远，相关性越小...')
                        vt /= np.sqrt(np.log2(distance+1))
                    if penalty2:
#                         print('对list长度惩罚，长度越长，对共现的价值越小...')
                        vt /= np.log10(len(items_)+9)
                    if i < j:
                        vt *= penalty3
                    mutex.acquire()
                    train_data_matrix[item2id.loc[item], item2id.loc[related_item]] += vt
                    mutex.release()
    pool.map(t, items_list)
    if penalty4:
        print('对item出现次数做惩罚...')
        def t(r):
#         for r in tqdm(range(train_data_matrix.shape[0])):
            for c in train_data_matrix.rows[r]:
                mutex.acquire()
                train_data_matrix[r,c] /= (np.log(item_num_dict[items[r]]+1)*np.log(item_num_dict[items[c]]+1))
                mutex.release()
        pool.map(t, range(train_data_matrix.shape[0]))
    if save_dir:
        if not os.path.exists(save_dir):
            print(f'create matrix dir {save_dir}')
            os.mkdir(save_dir)
        items.to_pickle(os.path.join(save_dir, f'id2item_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
        item2id.to_pickle(os.path.join(save_dir, f'item2id_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
        sp.save_npz(os.path.join(save_dir, f'item_item_matrix_{penalty1}_{penalty2}_{penalty3}.npz'), train_data_matrix.tocsc())
        print(f'save matrix to {save_dir}, finished')
    return train_data_matrix, items, item2id            

In [ ]:
# export
def recall_from_cocur_matr(matr, topk, user_pred_list, user_items, item2id, id2item):
    def t(u):
#         items = user_items[u][-30:]
        items = user_items[u]
        len_items = len(items)
#         print(len_items)
        sr = pd.Series()
        for i, item in enumerate(items):
#             sr = sr.append(pd.Series(matr[item2id[item]].toarray()[0]).sort_values()[-100:] * (len_items+i))
            sr = sr.append(pd.Series(matr[item2id[item]].toarray()[0]).sort_values()[-500:] * (len_items//2+1+i))
        sr = sr.reset_index()
        sr.columns = ['id', 'score']

        sr = sr.groupby('id')['score'].sum().reset_index()
        sr['item_id_pred'] = sr['id'].map(id2item)
        
        sr = sr[~sr.item_id_pred.isin(user_items[u])]
        sr = sr.sort_values('score', ascending=False)[:topk]
        sr['user_id'] = u
        sr['rank'] = range(1, sr.shape[0]+1)
#         rs.loc[u] = sr.sort_values('score', ascending=False)[:topk]['item_id score'.split()].values.tolist()
#         return u, ['item_id score'.split()].values.tolist()
        return sr['user_id item_id_pred score rank'.split()]
    rs = pd.Series()
    from multiprocessing.dummy import Pool
    pool = Pool(16)
    rs = pool.map(t, user_pred_list)
    df = pd.DataFrame()
    for d in rs:
        df = df.append(d)
#     rs = pd.Series(dict(rs))
    return df

In [ ]:
# export
def recall_from_bipartite_network(df,test_qtime, user_col, item_col):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
    
    item_user_ = df.groupby(item_col)[user_col].agg(set).reset_index()  
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))    

    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  

    sim_item = {}

    for item, users in tqdm(item_user_dict.items()):
    
        sim_item.setdefault(item, {}) 
    
        for u in users:
        
            tmp_len = len(user_item_dict[u])
        
            for relate_item in user_item_dict[u]:
                sim_item[item].setdefault(relate_item, 0)
                sim_item[item][relate_item] += 1/ (math.log(len(users)+1) * math.log(tmp_len+1))

    def recommend(sim_item_corr, user_item_dict, user_id, top_k):  
        rank = {}  
        interacted_items = user_item_dict[user_id]  
        for i in interacted_items:  
            for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:100]:  
                if j not in interacted_items:  
                    rank.setdefault(j, 0)  
                    rank[j] += wij  
        return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:top_k]  

    recom_item = []        
    for i in tqdm(test_qtime['user_id'].unique()):  
        rank_item = recommend(sim_item, user_item_dict , i, 500)  
        rank = 1
        for j in rank_item:  
            recom_item.append([i, j[0], j[1], rank])  # user_id, item_id, sim
            rank += 1
    
    dfr = pd.DataFrame(recom_item)

    dfr.columns = 'user_id item_id_pred score rank'.split()
    
#     dfr['item_id_score'] = dfr.item_id.map(lambda x: [x]) + dfr.score.map(lambda x: [x])
#     srr = dfr.sort_values('score', ascending=False).groupby('user_id')['item_id_score'].agg(list)
            
    return dfr

In [ ]:
# export
def recall_from_word2vec(
                        topk, user_pred_list, user_items, phase
                        ,dim=88
                        ,epochs=30
                        ,learning_rate=0.5, mode='train'
                        ):
    
    import psutil
    import gensim
    
    list_data = user_items.tolist()
#     print(len(list_data), list_data[:5])
    fp = os.path.join(f'../user_data/model_data/wvmodel_{mode}_{phase}')
    if os.path.exists(fp):
        print(f'load w2v..._{mode}_{phase}')
        model = gensim.models.Word2Vec.load(fp)
    else:
        print(f'train w2v..._{mode}_{phase}')
        model = gensim.models.Word2Vec(
                        list_data,
                        size=dim,
                        alpha=learning_rate,
                        window=999999,
                        min_count=1,
                        workers=psutil.cpu_count(),
                        compute_loss=True,
                        iter=epochs,
                        hs=0,
                        sg=1,
                        seed=42
                    )
        model.save(fp)
    print('------- word2vec 召回 ---------')
    def ft(u):  
        items = user_items[u]
        len_items = len(items)
        sr = pd.Series()
        for i, item in enumerate(items):
            try:
                sr = sr.append(pd.Series(dict(model.wv.most_similar(item, topn=500)))* (len_items//2+1+i))
            except:
                continue
        sr = sr.reset_index()
        sr.columns = ['item_id_pred', 'score']

        sr = sr.groupby('item_id_pred')['score'].sum().reset_index()
        
        sr = sr[~sr.item_id_pred.isin(user_items[u])]
        sr = sr.sort_values('score', ascending=False)[:topk]
        sr['user_id'] = u
        sr['rank'] = range(1, sr.shape[0]+1)
#         rs.loc[u] = sr.sort_values('score', ascending=False)[:topk]['item_id score'.split()].values.tolist()
#         return u, ['item_id score'.split()].values.tolist()
#         print(sr.shape)
        return sr['user_id item_id_pred score rank'.split()]
    rs = pd.Series()
    from multiprocessing.dummy import Pool
    pool = Pool(16)
    rs = pool.map(ft, user_pred_list)
    df = pd.DataFrame()
    for d in rs:
        df = df.append(d)
#     rs = pd.Series(dict(rs))
    return df

In [ ]:
# export
def t():
    phases = [7,8,9]
    r1 = []
    r2 = []
    r3 = []
    mode = 'test'
    window = 20
    for phase in phases:
        user_items = all_click_df[phase][0].append(all_click_df[phase][1]).groupby('user_id')['item_id'].agg(list)
        df_ = all_click_df[phase][0].append(all_click_df[phase][1])

        save_dir=None
        print(f'build cocur_matr {phase}')
        train_data_matrix, id2item, item2id = build_co_occurance_matrix(user_items.tolist(), window=window, 
                                                          penalty1=True, penalty2=True,penalty4=True,
                                                          penalty3=0.9, save_dir=save_dir)
        
        print(f'recall_from_cocur_matr {mode} {phase}')
        r1.append(recall_from_cocur_matr(train_data_matrix, 500, all_click_df[phase][2].user_id.tolist(), user_items, item2id, id2item))

        print(f'recall_from_word2vec {mode} {phase}')
        r2.append(recall_from_word2vec(
                        500, all_click_df[phase][2].user_id.tolist(), user_items,phase
                        ,dim=128
                        ,epochs=60
                        ,learning_rate=0.025, mode=mode
                        ))
        
        print(f'recall_from_bipartite_network {mode} {phase}')
        r3.append(recall_from_bipartite_network(df_,all_click_df[phase][2], 'user_id', 'item_id'))

    PickleWrapper.dump2File(r1, '../user_data/tmp_data/r_list_itemcf_test_789.pkl')
    PickleWrapper.dump2File(r2, '../user_data/tmp_data/r_list_w2v_test_789.pkl')
    PickleWrapper.dump2File(r3, '../user_data/tmp_data/r_list_binn_test_789.pkl')

# load match result

In [125]:
# export
def load_match_items789():
    r_itemcf = PickleWrapper.loadFromFile('../user_data/tmp_data/r_list_itemcf_test_789.pkl')

    r_binn = PickleWrapper.loadFromFile('../user_data/tmp_data/r_list_binn_test_789.pkl')
    
    r_itemcf_yl = PickleWrapper.loadFromFile('../user_data/tmp_data/r_list_itemcf_op_test_789.pkl')
    r_list_w2v = PickleWrapper.loadFromFile('../user_data/tmp_data/r_list_w2v_test_789.pkl')

    return r_itemcf, r_binn, r_itemcf_yl, r_list_w2v


In [128]:
# export

def data_preporcess(recall_list, match_num, phase, mode='train'):
    
    itemcf, r_binn, r_itemcf_yl, r_list_w2v = recall_list
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in range(len(phase)):
        r_itemcf[i]['phase'] = phase[i]
        r_binn[i]['phase'] = phase[i]
        r_itemcf_yl[i]['phase'] = phase[i]
        r_list_w2v[i]['phase'] = phase[i]
        df1 = df1.append(r_itemcf[i])
        df2 = df2.append(r_binn[i])
        df3 = df3.append(r_itemcf_yl[i])
        df4 = df4.append(r_list_w2v[i])

    df1_ = df1[df1['rank']<(match_num+1)]
    df2_ = df2[df2['rank']<(match_num+1)]
    df3_ = df3[df3['rank']<(match_num+1)]
    df4_ = df4[df4['rank']<(match_num+1)]

    print('merge Multi-channel recall...')
    cols = 'user_id item_id_pred phase'.split()
    
    if mode == 'train':
        cols = 'user_id item_id_pred item_id_true phase'.split()
    df = pd.merge(pd.merge(pd.merge(df1_, 
                                    df2_, on=cols, how='outer'), 
                                      df3_, on=cols, how='outer'), 
                                      df4_, on=cols, how='outer')

    df = df.fillna(0)


    dft = pd.DataFrame()
    for p in tqdm(phase):
        temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
        temp_ = pd.DataFrame(temp_)
        for i in range(0, 3):
            temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        dft = dft.append(temp_.reset_index())
        t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
        df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

    df = pd.merge(df, dft.drop(columns='item_id'))
    df.columns = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3']
    df['item_id_pred_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df['item_id_pred_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

    for c in tqdm('last_1 last_2 last_3'.split()):
        df[f'{c}_text_vec'] = df[c].map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

        df[f'{c}_img_vec'] = df[c].map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))


    return df

# FE

In [151]:
# export
def fe(df):
    for i in tqdm('rank score'.split()):
        df[f'{i}1_sub_{i}2'] = df[f'{i}_1']-df[f'{i}_2']
        df[f'{i}1_add_{i}2'] = df[f'{i}_1']+df[f'{i}_2']
        df[f'{i}1_mul_{i}2'] = df[f'{i}_1']*df[f'{i}_2']

        df[f'{i}1_sub_{i}3'] = df[f'{i}_1']-df[f'{i}_3']
        df[f'{i}1_add_{i}3'] = df[f'{i}_1']+df[f'{i}_3']
        df[f'{i}1_mul_{i}3'] = df[f'{i}_1']*df[f'{i}_3']

        df[f'{i}1_sub_{i}4'] = df[f'{i}_1']-df[f'{i}_4']
        df[f'{i}1_add_{i}4'] = df[f'{i}_1']+df[f'{i}_4']
        df[f'{i}1_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_4']

        df[f'{i}2_sub_{i}3'] = df[f'{i}_2']-df[f'{i}_3']
        df[f'{i}2_add_{i}3'] = df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}2_mul_{i}3'] = df[f'{i}_2']*df[f'{i}_3']

        df[f'{i}2_sub_{i}4'] = df[f'{i}_2']-df[f'{i}_4']
        df[f'{i}2_add_{i}4'] = df[f'{i}_2']+df[f'{i}_4']
        df[f'{i}2_mul_{i}4'] = df[f'{i}_2']*df[f'{i}_4']

        df[f'{i}3_sub_{i}4'] = df[f'{i}_3']-df[f'{i}_4']
        df[f'{i}3_add_{i}4'] = df[f'{i}_3']+df[f'{i}_4']
        df[f'{i}3_mul_{i}4'] = df[f'{i}_3']*df[f'{i}_4']
        
        df[f'{i}1_add_{i}2_add_{i}3'] = df[f'{i}_1']+df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}1_mul_{i}2_mul_{i}3'] = df[f'{i}_1']*df[f'{i}_2']*df[f'{i}_3']
        
        df[f'{i}1_add_{i}2_add_{i}4'] = df[f'{i}_1']+df[f'{i}_2']+df[f'{i}_4']
        df[f'{i}1_mul_{i}2_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_2']*df[f'{i}_4']
        
        df[f'{i}4_add_{i}2_add_{i}3'] = df[f'{i}_4']+df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}4_mul_{i}2_mul_{i}3'] = df[f'{i}_4']*df[f'{i}_2']*df[f'{i}_3']
        
        df[f'{i}1_add_{i}2_add_{i}3_add_{i}4'] = df[f'{i}_1']+df[f'{i}_2']+df[f'{i}_3']+df[f'{i}_4']
        df[f'{i}1_mul_{i}2_mul_{i}3_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_2']*df[f'{i}_3']*df[f'{i}_4']
        

    df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last_1_text_vec']).map(sum)

    df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last_1_img_vec']).map(sum)

    df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last_2_text_vec']).map(sum)
    df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last_2_img_vec']).map(sum)

    df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last_3_text_vec']).map(sum)
    df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last_3_img_vec']).map(sum)
    
    df['sim1_text_img'] = df['sim1_text'] *  df['sim1_img']
    df['sim2_text_img'] = df['sim2_text'] *  df['sim2_img']
    df['sim3_text_img'] = df['sim3_text'] *  df['sim3_img']

    df['sim12_text'] = df['sim1_text'] + df['sim2_text']
    df['sim123_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text']

    df['sim12_img'] = df['sim1_img'] + df['sim2_img']
    df['sim123_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img']

    df['sim12_text_img'] = df['sim1_text_img'] + df['sim2_text_img']
    df['sim123_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img']
    
    return df

# model

In [14]:
#export
def get_model_input(df, mode='train'):
    from catboost import Pool, CatBoostClassifier

    cols = ['score_1', 'rank_1',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4',
       'item_cnt', 'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img',
       'rank1_add_rank2_add_rank3', 'rank1_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank4', 'rank1_mul_rank2_mul_rank4',
       'rank4_add_rank2_add_rank3', 'rank4_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank3_add_rank4',
       'rank1_mul_rank2_mul_rank3_mul_rank4', 'score1_add_score2_add_score3',
       'score1_mul_score2_mul_score3', 'score1_add_score2_add_score4',
       'score1_mul_score2_mul_score4', 'score4_add_score2_add_score3',
       'score4_mul_score2_mul_score3',
       'score1_add_score2_add_score3_add_score4',
       'score1_mul_score2_mul_score3_mul_score4']
    df['fscore'] = 0
    df['frank'] = 999
    cat_cols = []
    X_test = df[cols]
    test_data = Pool(data=X_test, cat_features=cat_cols) 
    return test_data, cat_cols

In [15]:
# export
def get_model():
    from catboost import Pool, CatBoostClassifier
    params = {
        'iterations': 20,
        'learning_rate': 0.1,
        'random_seed': 144,
        'custom_metric': 'F1',
        'loss_function': 'Logloss',
        'class_weights': [1, 20],
        }
    print(params)
    model = CatBoostClassifier(**params)
    return model

# main

In [5]:
# export
if __name__ == '__main__':
    match_num, phase = 500, [7,8,9]
    all_click_df = load_click_data_per_phase(9)
    t()
    item_feat = get_item_data()
    item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
    item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

    item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

    item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

    item_feat.set_index('item_id', inplace=True)
    
    r_itemcf, r_binn, r_itemcf_yl, r_list_w2v = load_match_items789()
    
    recall_list = [r_itemcf, r_binn, r_itemcf_yl, r_list_w2v]
    df = data_preporcess(recall_list, match_num, phase, mode='predict')
    dffp = fe(df)
        
    test_data, cat_cols = get_model_input(dffp, mode='predict')
    

    model = CatBoostClassifier()
    model.load_model(os.path.join('../user_data/model_data/model.dump'))
    dffp.loc[:, 'fscore'] = model.predict_proba(test_data)[:, 1]

    dffp.loc[:, 'frank'] = dffp.groupby('user_id')['fscore'].rank(method='first', ascending=False)
    df1 = dffp.sort_values(['user_id', 'fscore'], ascending=False)

    sub = df1[df1.frank<51]

    sub = sub.groupby('user_id')['item_id_pred'].agg(list)
    sub1 = pd.DataFrame()

    for i in range(50):
        sub1[f'c{i}'] = sub.str.get(i)

    sub1.applymap(int).to_csv('../prediction_result/result.csv', header=False)        

2020-06-14 13:31:09.212 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 0
2020-06-14 13:31:10.614 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-06-14 13:31:10.617 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 1
2020-06-14 13:31:11.898 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-06-14 13:31:11.901 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 2
2020-06-14 13:31:13.105 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-06-14 13:31:13.107 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 3
2020-06-14 13:31:14.674 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-06-14 13:31:14.677 | INFO     | code.ed